In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqllite://Users/aileen/Downloads//Starter_Code 23/Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.metadata.create_all(engine)

In [ ]:
# View all of the classes that automap found
classes = Base.classes.keys()
classes

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(func.max(measurement.date)).scalar()

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
one_year_ago = datetime.strptime(most_recent_date) - timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(measurement.date, measurement.prcp) \
    .filter(measurement.date >= one_year_ago) \
    .all()

# Save the query results as a Pandas DataFrame. Explicitly set the column names
precipitation_df = pd.DataFrame(precipitation_data, columns=['Date', 'Precipitation'])

# Sort the dataframe by date
precipitation_df = precipitation_df.sort_values(by='Date')

# Use Pandas Plotting with Matplotlib to plot the data
precipitation_df.plot(x='Date', y='Precipitation', figsize=(10, 6),)
plt.xlabel('Date')
plt.ylabel('Iinches')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
precipitation_stats = precipitation_df['Precipitation'].describe()
print(precipitation_stats)

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
total_stations = session.query(func.count(func.distinct(station.station))).scalar()
print(total_stations)

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
active_stations = session.query(measurement.station, func.count(measurement.station)) \
    .group_by(measurement.station) \
    .order_by(func.count(measurement.station).desc()) \
    .all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temperature_stats = session.query(func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)) \
    .filter(measurement.station == active_stations) \
    .first()

lowest_temp, highest_temp, avg_temp = temperature_stats

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

tobs_data = session.query(measurement.date, measurement.tobs) \
    .filter(measurement.station == active_stations) \
    .filter(measurement.date >= one_year_ago) \
    .all()

# Save the query results as a Pandas DataFrame with specific column names
tobs_df = pd.DataFrame(tobs_data, columns=['Date', 'Temperature'])

# Plot the results as a histogram
tobs_df.plot(kind='hist', y='Temperature', bins=12, figsize=(10, 6))
plt.xlabel('Temperature (°F)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()